# 📝 Exercise M1.04

The goal of this exercise is to evaluate the impact of using an arbitrary
integer encoding for categorical variables along with a linear classification
model such as Logistic Regression.

To do so, let's try to use `OrdinalEncoder` to preprocess the categorical
variables. This preprocessor is assembled in a pipeline with
`LogisticRegression`. The generalization performance of the pipeline can be
evaluated by cross-validation and then compared to the score obtained when
using `OneHotEncoder` or to some other baseline score.

First, we load the dataset.

In [1]:
import pandas as pd

adult_census = pd.read_csv("../datasets/adult-census.csv")

In [5]:
# split data into features/target
target_name = "class"
target = adult_census[target_name]
data = adult_census.drop(columns=[target_name, "education-num"])


In the previous notebook, we used `sklearn.compose.make_column_selector` to
automatically select columns with a specific data type (also called `dtype`).
Here, we use this selector to get only the columns containing strings (column
with `object` dtype) that correspond to categorical features in our dataset.

In [10]:
from sklearn.compose import make_column_selector as selector

# select just categorical data using make_column_selector(data)
categorical_columns_selector = selector(dtype_include=object)
categorical_columns = categorical_columns_selector(data)
data_categorical = data[categorical_columns]
data_categorical.head()

,workclass,education,marital-status,occupation,relationship,race,sex,native-country
0,Private,11th,Never-married,Machine-op-inspct,Own-child,Black,Male,United-States
1,Private,HS-grad,Married-civ-spouse,Farming-fishing,Husband,White,Male,United-States
2,Local-gov,Assoc-acdm,Married-civ-spouse,Protective-serv,Husband,White,Male,United-States
3,Private,Some-college,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,United-States
4,?,Some-college,Never-married,?,Own-child,White,Female,United-States


Define a scikit-learn pipeline composed of an `OrdinalEncoder` and a
`LogisticRegression` classifier.

Because `OrdinalEncoder` can raise errors if it sees an unknown category at
prediction time, you can set the `handle_unknown="use_encoded_value"` and
`unknown_value` parameters. You can refer to the [scikit-learn
documentation](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OrdinalEncoder.html)
for more details regarding these parameters.

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OrdinalEncoder
from sklearn.linear_model import LogisticRegression

# Write your code here.
log_pipeline = make_pipeline(
    OrdinalEncoder(handle_unknown="use_encoded_value",
                   unknown_value=-1),
    LogisticRegression(max_iter=500)
)

Your model is now defined. Evaluate it using a cross-validation using
`sklearn.model_selection.cross_validate`.

<div class="admonition note alert alert-info">
<p class="first admonition-title" style="font-weight: bold;">Note</p>
<p class="last">Be aware that if an error happened during the cross-validation,
<tt class="docutils literal">cross_validate</tt> would raise a warning and return NaN (Not a Number) as scores.
To make it raise a standard Python exception with a traceback, you can pass
the <tt class="docutils literal"><span class="pre">error_score="raise"</span></tt> argument in the call to <tt class="docutils literal">cross_validate</tt>. An
exception would be raised instead of a warning at the first encountered problem
and <tt class="docutils literal">cross_validate</tt> would stop right away instead of returning NaN values.
This is particularly handy when developing complex machine learning pipelines.</p>
</div>

In [28]:
from sklearn.model_selection import cross_validate

# Write your code here.
cv_results = cross_validate(log_pipeline, data_categorical, target, error_score="raise")

print(f"The mean CV score is {cv_results['test_score'].mean():.3f} ± {cv_results['test_score'].std():.3f}")

The mean CV score is 0.755 ± 0.002


Now, we would like to compare the generalization performance of our previous
model with a new model where instead of using an `OrdinalEncoder`, we use a
`OneHotEncoder`. Repeat the model evaluation using cross-validation. Compare
the score of both models and conclude on the impact of choosing a specific
encoding strategy when using a linear model.

In [30]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_validate

# Write your code here.
model = make_pipeline(
    OneHotEncoder(handle_unknown="ignore"), LogisticRegression(max_iter=500)
)

cv_results = cross_validate(model, data_categorical, target)
print(f"The mean CV score is {cv_results['test_score'].mean():.3f} ± {cv_results['test_score'].std():.3f}")

The mean CV score is 0.833 ± 0.003


With the linear classifier chosen, using an encoding that does not assume any
ordering lead to much better result.

The important message here is: linear model and `OrdinalEncoder` are used
together only for ordinal categorical features, i.e. features that have a
specific ordering. Otherwise, your model would perform poorly.